<a href="https://colab.research.google.com/github/jonkingseestheworld/raster_visualization/blob/main/WP_Burundi_folium_osm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## pip install GIS packages

!pip install rasterio
!pip install geopandas


In [ ]:
## import required packages
import rasterio
from rasterio import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
#import matplotlib as mlp
import re
%matplotlib inline

import numpy as np
import pandas as pd
import geopandas as gpd

import os
import folium

In [ ]:
## to mount google drive so that files can be automatically retrieved and saved there 
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
## change WD to the folder where I save the data/files in my google drive
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/satellite_analysis")

## (Only one-time code) Downloading Worldpop data 
(Doesn't save into my gdrive)

In [ ]:
### wget package does not exist in google colab
#!pip install wget

import wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=2e6af4250a0b5ad3fecc3cf6f0629f762214aa290de0af6f0a3c778f9534035e
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
### The code here is written for downloading data of a country from all years in a loop
### You can modify it to enable downloading for multiple countries in a go

country = "BDI"   ### update it to the country abbreviation needed

### create/specify the destination path where downloaded files are saved

#parent_dir = "/content/gdrive/MyDrive/Colab Notebooks/satellite_analysis/data/worldpop"
parent_dir = "/content"     # these files occupied too much of my gdrive. I cannot save them in my gdrive anymore

dest_path = os.path.join(parent_dir, "worldpop", country)
os.makedirs(dest_path, exist_ok=True) 


### Worldpop population data can be downloaded from their 'ftp' repo folder directly

for year in range(2020,2021):    # here to get data from year 2000 to 2020
  url = "ftp://ftp.worldpop.org.uk/GIS/Population/Global_2000_2020/%s/%s/%s_ppp_%s.tif" %(year, country, country, year)
  wget.download(url, dest_path)


# !wget ftp://ftp.worldpop.org.uk/GIS/Population/Global_2000_2020/2020/EGY/egy_ppp_2020.tif


In [ ]:
year=2020
#rast_img = 'data/worldpop/UGA/uga_ppp_{}.tif'.format(year)
rast_img = '/content/worldpop/BDI/BDI_ppp_{}.tif'.format(year)

In [ ]:
## function for reading a raster image (.tiff)
img = rasterio.open(rast_img)



In [ ]:
boundary = img.bounds
boundary

BoundingBox(left=29.000416566, bottom=-4.469583114, right=30.850416559, top=-2.308749789)

In [ ]:
c_lat = (boundary.bottom + boundary.top)/2
c_lon = (boundary.left + boundary.right)/2

In [ ]:
img.crs, img.indexes

(CRS.from_epsg(4326), (1,))

In [ ]:
# Create a map using Stamen Terrain, centered on Boulder, CO
m = folium.Map(location=[c_lat, c_lon], 
              tiles = 'OpenStreetMap', zoom_start = 7)

# Add marker for Boulder, CO
folium.Marker(
    location=[c_lat, c_lon], # coordinates for the marker (Earth Lab at CU Boulder)
    popup='BDI', # pop-up label for the marker
    icon=folium.Icon()
).add_to(m)

# Display m
m

In [ ]:
# Create a map using Stamen Terrain, centered on study area with set zoom level
m = folium.Map(location=[c_lat, c_lon],
                   tiles='Stamen Terrain', zoom_start = 10)

# Overlay raster called img using add_child() function (opacity and bounding box set)
m.add_child(folium.raster_layers.ImageOverlay(img.read(1), opacity=.7, 
                                 bounds =[[boundary.bottom, boundary.left], [boundary.top, boundary.right]]))




A spatial database of health facilities managed by the public health sector in sub Saharan Africa

https://www.nature.com/articles/s41597-019-0142-2

In [ ]:
hospitals_loc = pd.read_excel('who-cds-gmp-2019-01-eng.xlsx')

In [ ]:
hosp_bdi = hospitals_loc[(hospitals_loc.Country == "Burundi") & ((hospitals_loc.Admin1 == "Bujumbura Mairie") | (hospitals_loc.Admin1 == "Bujumbura Rural"))]
hosp_bdi.head()

,Country,Admin1,Facility name,Facility type,Ownership,Lat,Long,LL source
4764,Burundi,Bujumbura Mairie,Butere Centre de Santé,Health Centre,Public,-3.33083,29.34303,GPS
4765,Burundi,Bujumbura Mairie,Buterere 2 Centre de Santé,Health Centre,Public,-3.34001,29.34231,GPS
4766,Burundi,Bujumbura Mairie,Bwiza Jabe Centre de Santé,Health Centre,Public,-3.37088,29.37044,GPS
4767,Burundi,Bujumbura Mairie,Catb Bujumbura Centre de Santé,Health Centre,Public,-3.37277,29.35935,GPS
4768,Burundi,Bujumbura Mairie,Centre Communautaire Kamenge Centre de Santé,Health Centre,Public,-3.35534,29.38468,GPS


In [ ]:
hosp_bdi.Ownership.unique()

array(['Public', 'FBO'], dtype=object)

In [ ]:
hosp_types = hosp_bdi["Facility type"].unique()
type_1= hosp_bdi[hosp_bdi["Facility type"] == hosp_types[1]]

for i, type in enumerate(hosp_types):
  _hosp = hosp_bdi[hosp_bdi["Facility type"] == type]
  print(i, type)

0 Health Centre
1 Hôpital Tertiaire
2 Hôpital de District


In [ ]:
#Adding markers to the map

colors = ["red", "blue", "green"]

for i, type in enumerate(hosp_types):
  _hosp = hosp_bdi[hosp_bdi["Facility type"] == type]
  for lat, long in zip(_hosp.Lat, _hosp.Long):
    folium.Circle(location=[lat, long],radius=300, weight=5, tooltip= type, color=colors[i], fill=True,fill_color=colors[i]).add_to(m)


# Display map 
m